In [37]:
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import torch
import torch.nn as nn
from torchvision.models import resnet50
from torch.utils.data.dataloader import Dataset, DataLoader
from torch.optim.adam import Adam 
from torchvision import transforms
import torchvision
import cv2
from PIL import Image
import glob



In [38]:
# 윈도우 cuda 설정
'''
device = "cuda" if torch.cuda.is_available() else "cpu"
device
'''

'\ndevice = "cuda" if torch.cuda.is_available() else "cpu"\ndevice\n'

In [39]:
# 맥 mps 설정
device = torch.device("mps:0" if torch.backends.mps.is_available() else "cpu")
print(f"현재 디바이스는 {device} 입니다.")

현재 디바이스는 mps:0 입니다.


# 데이터 전처리

In [40]:
trans = transforms.Compose([
    transforms.ToTensor()
])

In [41]:
trainset = torchvision.datasets.ImageFolder(root = "data/x20/train_x20", transform=trans)

In [42]:
imgs = [item[0] for item in trainset]

In [43]:
imgs = torch.stack(imgs, dim=0).numpy()

In [44]:
# 데이터 정규화를 위한 평균값
mean_r = imgs[:,0,:,:].mean()
mean_g = imgs[:,1,:,:].mean()
mean_b = imgs[:,2,:,:].mean()
print(mean_r, mean_g, mean_b)

0.7904179 0.5446238 0.6990631


In [45]:
# 데이터 정규화를 위한 표준편차
std_r = imgs[:,0,:,:].std()
std_g = imgs[:,1,:,:].std()
std_b = imgs[:,2,:,:].std()
print(std_r, std_g, std_b)

0.17362285 0.2598456 0.1853092


In [46]:
trans = transforms.Compose([transforms.Resize((1500,800)),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=(mean_r,mean_g,mean_b),
                                                 std = (std_r,std_g,std_b))
                            ])

# 데이터 불러오기

## 트레인 데이터 불러오기

In [47]:
trainset = torchvision.datasets.ImageFolder(root = "data/x20/train_x20", transform=trans)

In [48]:
len(trainset)

50

In [49]:
classes = trainset.classes
classes

['non_tumor', 'tumor']

In [50]:
trainloader = DataLoader(trainset, batch_size = 4, shuffle = True)

In [51]:
images1, labels1 = next(iter(trainloader))
print(labels1)

tensor([0, 1, 0, 0])


# Validation 데이터 불러오기

In [52]:
validation = torchvision.datasets.ImageFolder(root = "data/x20/val_x20", transform=trans)

In [53]:
len(validation)

20

In [54]:
validloader = DataLoader(validation, batch_size = 4, shuffle = False)

## 테스트 데이터 불러오기

In [55]:
testset = torchvision.datasets.ImageFolder(root = "data/x20/test_x20", transform=trans)

In [56]:
len(testset)

30

In [57]:
classes = testset.classes
classes

['non_tumor', 'tumor']

In [58]:
testloader = DataLoader(testset, batch_size = 4, shuffle = False)

In [59]:
images2, labels2 = next(iter(testloader))
print(labels2)

tensor([0, 0, 0, 0])


# 모델 정의

In [60]:
model = resnet50(weights = None)

In [61]:
model.fc = nn.Sequential(
    nn.Linear(2048, 128),
    nn.ReLU(inplace = True),
    nn.Linear(128, 2))

In [62]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [63]:
# 학습 루프 정의
lr = 1e-4
optim = Adam(model.parameters(), lr=lr)

for epoch in range(20):
    iterator = tqdm.tqdm(trainloader)
    for data, label in iterator:
        model.train()
        optim.zero_grad()
        preds = model(data.to(device))

        loss = nn.CrossEntropyLoss()(preds, label.to(device))
        loss.backward()
        optim.step()
        
        iterator.set_description(f"epoch:{epoch+1} loss:{loss.item()}")

torch.save(model.state_dict(), "ResNet_test.pth")

epoch:1 loss:2.229645252227783:  69%|██████▉   | 9/13 [21:08<09:23, 140.90s/it] 


KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load("ResNet_test.pth", map_location=device))

num_corr = 0

model.eval()
with torch.no_grad():
    for data, label in testloader:
        output = model(data.to(device))
        preds = output.data.max(1)[1]
        corr = preds.eq(label.to(device).data).sum().item()
        num_corr += corr
    print (f"Accuracy : {num_corr/len(testset)}")

Accuracy : 0.6666666666666666
